<img src="https://raw.githubusercontent.com/maxsitt/insect-detect-docs/main/docs/assets/logo.png" width="500">

# YOLOv6 detection model training for deployment on Luxonis OAK

Author: &nbsp; Maximilian Sittinger &nbsp;
[<img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width="24">](https://github.com/maxsitt) &nbsp;
[<img src="https://upload.wikimedia.org/wikipedia/commons/0/06/ORCID_iD.svg" width="24">](https://orcid.org/0000-0002-4096-8556)

- 📑 [**Insect Detect Docs**](https://maxsitt.github.io/insect-detect-docs/)
- [`insect-detect-ml` GitHub repo](https://github.com/maxsitt/insect-detect-ml)

&nbsp;

**Train a [YOLOv6](https://github.com/meituan/YOLOv6) object detection model on your own custom training data!**

- Using dataset import from [Roboflow](https://roboflow.com/) is recommended, but is not required.
> Choose option *Upload dataset from Google Drive/local file system* instead (slower!).
- Connecting to Google Drive is recommended, but is not required.
> Choose options *Upload dataset from your local file system* and *Download* instead of *Export to Google Drive* (slower!).
- Go to **File** in the top menu bar and choose **Save a copy in Drive** before running the notebook.
- Go to **Runtime** and make sure that **GPU** is selected as Hardware accelerator under **Change runtime type**.
- If you are using Firefox, please make sure to allow notifications for this website.

&nbsp;

---

**References**

1. Official YOLOv6 tutorial notebook by MeiTuan &emsp;
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meituan/YOLOv6/blob/main/turtorial.ipynb)
2. Roboflow tutorial notebook for YOLOv6 training &nbsp;
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/notebooks/blob/main/notebooks/train-yolov6-object-detection-on-custom-data.ipynb)
3. DepthAI tutorial notebook for YOLOv6 training &emsp;
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/luxonis/depthai-ml-training/blob/master/colab-notebooks/YoloV6_training.ipynb)

# Initialization

## Show GPU + Linux distribution

In [ ]:
!nvidia-smi -L
print("\n")
!head -n 2 /etc/*release

## YOLOv6 setup

In [ ]:
!git clone https://github.com/meituan/YOLOv6.git
%cd YOLOv6
%pip install -qr requirements.txt

## Recommended: Upload dataset from Roboflow

If you are not sure how to export your annotated dataset in YOLOv6 format, check the [Roboflow docs](https://docs.roboflow.com/exporting-data).

> Alternatively you can upload your dataset ([YOLOv6 format](https://github.com/meituan/YOLOv6/blob/main/docs/Train_custom_data.md#1-prepare-your-own-dataset)) from **[Google Drive](#scrollTo=RxOnnOadc5vR)** or from your **[local file system](#scrollTo=qKTCWdtkOUw7)** in the next steps!

In [ ]:
%pip install -q roboflow==0.2.34
from roboflow import Roboflow
rf = Roboflow(model_format = "yolov6", notebook = "insdet_yolov6")

**Copy only the last three lines of the Download Code and insert them at the top of the next code cell:**

In [ ]:
### Paste your Download Code here:
rf = Roboflow(api_key="XXXXXXXXXXXXXXXXXXXX")
project = rf.workspace("maximilian-sittinger").project("insect_detect_detection")
dataset = project.version(7).download("mt-yolov6")
###

dataset_location = dataset.location

from pathlib import Path
print(f"\nLocation of dataset: {dataset_location}\n")
print("Number of training images:", len(list(Path(f"{dataset_location}/images/train").glob("*.jpg"))))
print("Number of validation images:", len(list(Path(f"{dataset_location}/images/valid").glob("*.jpg"))))
print("Number of test images:", len(list(Path(f"{dataset_location}/images/test").glob("*.jpg"))))
print("\nContent of data.yaml file:\n")
%cat {dataset_location}/data.yaml

## Recommended: Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
#@title ## Optional: Upload dataset from Google Drive {display-mode: "form"}

#@markdown ### Google Drive path to dataset folder:
dataset_path = "MyDrive/Datasets/yolov6_dataset" #@param {type: "string"}

%cp -ai /content/drive/{dataset_path} /content/YOLOv6

from pathlib import Path
dataset_name = Path(dataset_path).stem
dataset_location = f"/content/YOLOv6/{dataset_name}"

print(f"Location of dataset: {dataset_location}\n")
print("Number of training images:", len(list(Path(f"{dataset_location}/images/train").glob("*.jpg"))))
print("Number of validation images:", len(list(Path(f"{dataset_location}/images/valid").glob("*.jpg"))))
print("Number of test images:", len(list(Path(f"{dataset_location}/images/test").glob("*.jpg"))))
print("\nContent of data.yaml file:\n")
%cat {dataset_location}/data.yaml

In [ ]:
#@title ## Optional: Upload dataset from your local file system {display-mode: "form"}

#@markdown ### Name of your (zipped) dataset folder:
dataset_name = "yolov6_dataset" #@param {type: "string"}
#@markdown - Please make sure to compress your dataset folder to **.zip file** before uploading!
#@markdown - The name of the .zip file should be the same as for the dataset folder.
#@markdown - Dataset has to be in [YOLOv6 format](https://github.com/meituan/YOLOv6/blob/main/docs/Train_custom_data.md#1-prepare-your-own-dataset).

dataset_location = f"/content/YOLOv6/{dataset_name}"

from google.colab import files
uploaded = files.upload()

import zipfile
if len(list(zipfile.Path(f"{dataset_name}.zip").iterdir())) > 1:
  !unzip -uq {dataset_name}.zip -d /content/YOLOv6/{dataset_name}
else:
  !unzip -uq {dataset_name}.zip -d /content/YOLOv6
%rm {dataset_name}.zip

from pathlib import Path
print(f"\nLocation of dataset: {dataset_location}\n")
print("Number of training images:", len(list(Path(f"{dataset_location}/images/train").glob("*.jpg"))))
print("Number of validation images:", len(list(Path(f"{dataset_location}/images/valid").glob("*.jpg"))))
print("Number of test images:", len(list(Path(f"{dataset_location}/images/test").glob("*.jpg"))))
print("\nContent of data.yaml file:\n")
%cat {dataset_location}/data.yaml

## Optional: Edit `data.yaml`

If you chose to upload your training dataset from **Google Drive** or your **local file system**, you have to check the `data.yaml` file in your dataset folder to make sure the paths to the train, valid and test folders are correct.

- Open your dataset folder in the File Explorer (Folder symbol on the left side bar).
- Double-click on the `data.yaml` file, it will open in the editor to the right.

  Make sure that the paths to the train, valid and test folders are as follows:

  ```
  train: <YOUR_DATASET_NAME>/images/train
  val: <YOUR_DATASET_NAME>/images/valid
  test: <YOUR_DATASET_NAME>/images/test
  ```

- Insert the correct name of your dataset folder at `<YOUR_DATASET_NAME>`.
- Save your changes with **Ctrl + S** and close the editor.

# Model training

## Tensorboard logger

> If you are using Firefox, **disable Enhanced Tracking Protection** for this website (click on the shield to the left of the address bar) for the Tensorboard logger to work correctly!

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/YOLOv6/runs/train

## Train YOLOv6 detection model

- `--name` name of the training run
- `--img` input image size (recommended: same size as for inference)
- `--batch` specify batch size (recommended: 32)
- `--epochs` set the number of training [epochs](https://machine-learning.paperspace.com/wiki/epoch) (recommended: 100-300+ epochs)
- `--data` path to `data.yaml` file
- `--conf` specify the model configuration file ([pretrained model weights](https://github.com/meituan/YOLOv6#benchmark))
> `--conf configs/yolov6n_finetune.py` YOLOv6-nano model (recommended)  
  `--conf configs/yolov6s_finetune.py` YOLOv6-small model

> More information on YOLOv6 [model training](https://blog.roboflow.com/how-to-train-yolov6-on-a-custom-dataset/#configure-yolov6-custom-training-options) 🚀

In [ ]:
training_run_name = "YOLOv6n_320_batch32_epochs300" #@param {type: "string"}
#@markdown **Add UTC timestamp in front of training run name:**
add_timestamp = True #@param {type:"boolean"}
#@markdown ---

image_size = 320 #@param {type: "integer"}
batch_size = 32 #@param {type:"slider", min:16, max:128, step:16}
number_epochs = 300 #@param {type:"slider", min:10, max:600, step:10}
config = "yolov6n_finetune.py" #@param ["yolov6n_finetune.py", "yolov6s_finetune.py"]

if add_timestamp == True:
  from datetime import datetime
  utc_timestamp = datetime.now().strftime("%Y%m%d_%H-%M")
  train_run_name = f"{utc_timestamp}_{training_run_name}"
else:
  train_run_name = training_run_name

%cd /content/YOLOv6

!python tools/train.py \
--name {train_run_name} \
--img {image_size} \
--batch {batch_size} \
--epochs {number_epochs} \
--data {dataset_location}/data.yaml \
--conf configs/{config}

In [ ]:
#@title ## Export to Google Drive or Download training results {display-mode: "form"}

training_results = "Export_Google_Drive" #@param ["Export_Google_Drive", "Download"]
#@markdown ---

#@markdown ### Path for saving training results in Google Drive:
GDrive_save_path = "MyDrive/Training_results/YOLOv6"  #@param {type: "string"}

if training_results == "Export_Google_Drive":
  %mkdir -p /content/drive/{GDrive_save_path}
  %cp -ai /content/YOLOv6/runs/train/{train_run_name} /content/drive/{GDrive_save_path}
elif training_results == "Download":
  %cd /content/YOLOv6/runs/train
  !zip -rq {train_run_name}.zip {train_run_name}
  from google.colab import files
  files.download(f"{train_run_name}.zip")

# Model validation

Check the performance of your model on the dataset valid split.

> Copy the validation results (cell output) and save to .txt file, as they will not be saved automatically.

Use `--task test` to validate on the dataset test split.

In [ ]:
%cd /content/YOLOv6

!python tools/eval.py \
--name {train_run_name}_validate \
--weights runs/train/{train_run_name}/weights/best_ckpt.pt \
--data {dataset_location}/data.yaml \
--img {image_size} \
--do_pr_metric True \
--plot_confusion_matrix \
#--task test

In [ ]:
#@title ## Export to Google Drive or Download validation results {display-mode: "form"}

validation_results = "Download" #@param ["Export_Google_Drive", "Download"]
#@markdown ---

#@markdown ### Path for saving validation results in Google Drive:
GDrive_save_path = "MyDrive/Training_results/YOLOv6"  #@param {type: "string"}

if validation_results == "Export_Google_Drive":
  %mkdir -p /content/drive/{GDrive_save_path}
  %cp -ai /content/YOLOv6/runs/val/{train_run_name}_validate /content/drive/{GDrive_save_path}
elif validation_results == "Download":
  %cd /content/YOLOv6/runs/val
  !zip -rq {train_run_name}_validate.zip {train_run_name}_validate
  from google.colab import files
  files.download(f"{train_run_name}_validate.zip")

# Model inference

Test the detection accuracy of your model on the dataset test split.

In [ ]:
#@markdown #### Decrease confidence threshold to detect objects with lower confidence score:
confidence_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}
#@markdown #### Increase IoU threshold if the same object is detected multiple times:
iou_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}

%cd /content/YOLOv6

!python tools/infer.py \
--name {train_run_name}_detect \
--weights runs/train/{train_run_name}/weights/best_ckpt.pt \
--yaml {dataset_location}/data.yaml \
--source {dataset_location}/images/test/ \
--img {image_size} {image_size} \
--conf-thres {confidence_threshold} \
--iou-thres {iou_threshold}

## Show inference results on test images

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob(f"/content/YOLOv6/runs/inference/{train_run_name}_detect/*.jpg"):
  display(Image(filename=imageName))
  print("\n")

# Model conversion

**Go to https://tools.luxonis.com/:**

- Select `YoloV6` as Yolo Version.
- Rename your model weights file from `best_ckpt.pt` to e.g. `yolov6n320.pt`.
- Select your model weights file (`yolov6n320.pt`) for upload.
- Use your image size (e.g. `320`) as input shape.
- Open the `Advanced options` and choose `Shaves: 4`.
- Hit `Submit` to upload your model weights and download the converted ONNX, OpenVINO and .blob models.

**Recommended number of shaves the model can use is 4-5 for the Insect Detect camera trap!**

> More information on model conversion can be found at the [DepthAI docs](https://docs.luxonis.com/en/latest/pages/model_conversion/).

> More information on the number of shaves can be found at the [DepthAI FAQ](https://docs.luxonis.com/en/latest/pages/faq/#what-are-the-shaves).

## Generate JSON config file

Together with the converted model, a .json config file with model specific settings will be created after following the conversion steps at https://tools.luxonis.com/.

> To set the correct class/label name(s) and adjust the confidence or IoU threshold, you can create and download your own .json config file in the following step (or change it directly in the .json file you downloaded from tools.luxonis.com).

In [ ]:
#@markdown ### Name for the JSON config file:
json_name = "yolov6_320" #@param {type: "string"}
#@markdown ---

image_size = 320 #@param {type: "integer"}
number_classes = 1 #@param {type: "integer"}
#@markdown ---

#@markdown #### For several classes/labels: **["class1", "class2", "class3"]**
labels = ["insect"] #@param {type: "raw"}
#@markdown ---

#@markdown #### Decrease confidence threshold to detect objects with lower confidence score:
confidence_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}
#@markdown #### Increase IoU threshold if the same object is detected multiple times:
iou_threshold = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}

!wget -L https://raw.githubusercontent.com/luxonis/depthai-experiments/master/gen2-yolo/device-decoding/json/yolov5.json -P /content/

import json
with open("/content/yolov5.json", "r") as f:
  json_data = json.load(f)
  json_data["nn_config"]["input_size"] = f"{image_size}x{image_size}"
  json_data["nn_config"]["NN_specific_metadata"]["classes"] = number_classes
  json_data["nn_config"]["NN_specific_metadata"]["anchors"] = []
  json_data["nn_config"]["NN_specific_metadata"]["anchor_masks"] = {}
  json_data["nn_config"]["NN_specific_metadata"]["iou_threshold"] = iou_threshold
  json_data["nn_config"]["NN_specific_metadata"]["confidence_threshold"] = confidence_threshold
  json_data["mappings"]["labels"] = labels

with open(f"/content/{json_name}.json", "w") as f:
  json.dump(json_data, f, indent = 4)

from google.colab import files
files.download(f"/content/{json_name}.json")

# Model deployment

That's it! You trained your own [YOLOv6](https://github.com/meituan/YOLOv6) object detection model with your custom dataset and converted it to .blob format which is necessary to run inference on the [Luxonis OAK devices](https://docs.luxonis.com/projects/hardware/en/latest/).

> To deploy the YOLOv6 model on your OAK you can check out the Luxonis GitHub repository for [on-device decoding](https://github.com/luxonis/depthai-experiments/tree/master/gen2-yolo/device-decoding) or use the deployment options from the [**Insect Detect Docs**](https://maxsitt.github.io/insect-detect-docs/software/programming/) (e.g. for continuous automated insect monitoring with the DIY camera trap).